In [1]:
import tkinter as tk
from tkinter import ttk
import mysql.connector
from tkinter import messagebox

# Connect to MySQL database
connection = mysql.connector.connect(
    host="localhost",
    port=3306,
    user="root",
    password="root",
    database="company"
)

# Function to retrieve and display data of a selected table
def display_table_data(event):
    selected_item = tree.focus()
    table_name = tree.item(selected_item)['values'][0]
    cursor = connection.cursor()
    cursor.execute(f"SELECT * FROM {table_name}")
    columns = [column[0] for column in cursor.description]
    data = cursor.fetchall()
    cursor.close()
    
    # Create a new window for displaying table data
    table_window = tk.Toplevel(root)
    table_window.title(table_name)
    
    # Create a treeview with scrollbar
    table_tree = ttk.Treeview(table_window)
    table_tree.pack(expand=True, fill=tk.BOTH)
    vsb = ttk.Scrollbar(table_window, orient="vertical", command=table_tree.yview)
    vsb.pack(side=tk.RIGHT, fill=tk.Y)
    table_tree.configure(yscrollcommand=vsb.set)
    
    # Configure treeview columns
    table_tree["columns"] = columns
    table_tree.heading("#0", text="Row")
    table_tree.column("#0", width=50)
    for column in columns:
        table_tree.heading(column, text=column)
        table_tree.column(column, width=100)
    
    # Insert data into the treeview
    for i, row in enumerate(data):
        table_tree.insert("", tk.END, text=str(i+1), values=row)

# Function to retrieve and display all tables in the schema
def display_tables():
    clear_table()
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES")
    tables = [table[0] for table in cursor.fetchall()]
    tree["columns"] = ("Table Name",)
    tree.heading("#0", text="Tables in Schema", anchor=tk.W)
    tree.column("#0", anchor=tk.W)
    tree.heading("Table Name", text="Table Name")
    tree.column("Table Name", anchor=tk.W)
    for table in tables:
        tree.insert("", tk.END, text="", values=(table,))
    cursor.close()

# Function to clear the table
def clear_table():
    tree.delete(*tree.get_children())

# Function to handle the "Queries" button
def open_queries_screen():
    queries_window = tk.Toplevel(root)
    queries_window.title("Queries")

    # Create a frame for the buttons
    button_frame = tk.Frame(queries_window)
    button_frame.pack(pady=10)

    # Create a button for the location query
    location_button = ttk.Button(button_frame, text="Location Query", command=execute_location_query_window)
    location_button.pack(side=tk.LEFT, padx=5)

    # Create a button for the salary query
    salary_button = ttk.Button(button_frame, text="Salary Query", command=execute_salary_query_window)
    salary_button.pack(side=tk.LEFT, padx=5)

def execute_location_query_window():
    query_window = tk.Toplevel(root)
    query_window.title("Location Query")
    
    # Create a label and entry for the location input
    location_label = tk.Label(query_window, text="Location:")
    location_label.pack(pady=10)
    location_entry = tk.Entry(query_window)
    location_entry.pack(pady=5)
    
    # Create a button to execute the location query
    execute_button = ttk.Button(query_window, text="Execute Location Query", command=lambda: execute_location_query(location_entry.get()))
    execute_button.pack(pady=10)

def execute_salary_query_window():
    query_window = tk.Toplevel(root)
    query_window.title("Salary Query")
    
    # Create a label and entry for the salary input
    salary_label = tk.Label(query_window, text="Salary:")
    salary_label.pack(pady=10)
    salary_entry = tk.Entry(query_window)
    salary_entry.pack(pady=5)
    
    # Create a button to execute the salary query
    execute_button = ttk.Button(query_window, text="Execute Salary Query", command=lambda: execute_salary_query(salary_entry.get()))
    execute_button.pack(pady=10)

def execute_location_query(location):
    cursor = connection.cursor()
    cursor.execute("""
        SELECT E.*
        FROM Employee AS E
        JOIN Employee_position AS EP ON E.EmployeeID = EP.EmployeeID
        JOIN Department AS D ON EP.Department_no = D.Departmentno
        JOIN Department_location AS DL ON D.Departmentno = DL.Departmentno
        WHERE DL.DepartmentLocation = %s
    """, (location,))
    columns = [column[0] for column in cursor.description]
    data = cursor.fetchall()
    cursor.close()

    # Create a new window for displaying query results
    query_window = tk.Toplevel(root)
    query_window.title("Query Results")

    # Create a treeview with scrollbar
    query_tree = ttk.Treeview(query_window)
    query_tree.pack(expand=True, fill=tk.BOTH)
    vsb = ttk.Scrollbar(query_window, orient="vertical", command=query_tree.yview)
    vsb.pack(side=tk.RIGHT, fill=tk.Y)
    query_tree.configure(yscrollcommand=vsb.set)

    # Configure treeview columns
    query_tree["columns"] = columns
    query_tree.heading("#0", text="Row")
    query_tree.column("#0", width=50)
    for column in columns:
        query_tree.heading(column, text=column)
        query_tree.column(column, width=100)

    # Insert data into the treeview
    for i, row in enumerate(data):
        query_tree.insert("", tk.END, text=str(i + 1), values=row)

def execute_salary_query(salary):
    try:
        salary = float(salary)
        cursor = connection.cursor()
        cursor.execute("""
            SELECT *
            FROM Employee
            WHERE EmployeeID IN (
                SELECT EmployeeID
                FROM Employee_Salary
                WHERE salary > %s
            )
        """, (salary,))
        columns = [column[0] for column in cursor.description]
        data = cursor.fetchall()
        cursor.close()

        # Create a new window for displaying query results
        query_window = tk.Toplevel(root)
        query_window.title("Query Results")

        # Create a treeview with scrollbar
        query_tree = ttk.Treeview(query_window)
        query_tree.pack(expand=True, fill=tk.BOTH)
        vsb = ttk.Scrollbar(query_window, orient="vertical", command=query_tree.yview)
        vsb.pack(side=tk.RIGHT, fill=tk.Y)
        query_tree.configure(yscrollcommand=vsb.set)

        # Configure treeview columns
        query_tree["columns"] = columns
        query_tree.heading("#0", text="Row")
        query_tree.column("#0", width=50)
        for column in columns:
            query_tree.heading(column, text=column)
            query_tree.column(column, width=100)

        # Insert data into the treeview
        for i, row in enumerate(data):            
            query_tree.insert("", tk.END, text=str(i + 1), values=row)

    except ValueError:
        # Handle the case when the user enters an invalid salary value
        tk.messagebox.showerror("Error", "Invalid salary value. Please enter a valid number.")

# Function to handle the "Insert" button
def open_insert_screen():
    insert_window = tk.Toplevel(root)
    insert_window.title("Insert Data")
    
    # Create a label and entry for the table name input
    table_label = tk.Label(insert_window, text="Table Name:")
    table_label.pack(pady=10)
    table_entry = tk.Entry(insert_window)
    table_entry.pack(pady=5)
    
    # Create a button to open the insert data window
    execute_button = ttk.Button(insert_window, text="Open Insert Data Window", command=lambda: open_insert_data_screen(table_entry.get()))
    execute_button.pack(pady=10)

def open_insert_data_screen(table_name):
    insert_data_window = tk.Toplevel(root)
    insert_data_window.title("Insert Data")
    
    # Fetch columns for the selected table
    cursor = connection.cursor()
    cursor.execute(f"SHOW COLUMNS FROM {table_name}")
    columns = [column[0] for column in cursor.fetchall()]
    cursor.close()
    
    # Create labels and entries for each column
    entry_list = []
    for column in columns:
        label = tk.Label(insert_data_window, text=column + ":")
        label.pack(pady=5)
        entry = tk.Entry(insert_data_window)
        entry.pack(pady=5)
        entry_list.append(entry)
    
    # Create a button to execute the insert query
    execute_button = ttk.Button(insert_data_window, text="Execute Insert Query", command=lambda: execute_insert_query(table_name, columns, entry_list))
    execute_button.pack(pady=10)

def execute_insert_query(table_name, columns, entry_list):
    # Collect values from entry fields
    values = [entry.get() for entry in entry_list]
    
    # Construct the SQL query
    query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"
    
    # Execute the insert query
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    cursor.close()
    
    # Show a success message
    tk.messagebox.showinfo("Success", "Data inserted successfully.")

# Create the main window
root = tk.Tk()
root.title("Table Viewer")

# Create a treeview with scrollbar
tree = ttk.Treeview(root)
tree.pack(expand=True, fill=tk.BOTH)
vsb = ttk.Scrollbar(root, orient="vertical", command=tree.yview)
vsb.pack(side=tk.RIGHT, fill=tk.Y)
tree.configure(yscrollcommand=vsb.set)

# Bind double-click event to table names
tree.bind("<Double-1>", display_table_data)

# Create a button to display tables
tables_button = ttk.Button(root, text="Display Tables", command=display_tables)
tables_button.pack(pady=5)

# Create a button to open the queries screen
queries_button = ttk.Button(root, text="Queries", command=open_queries_screen)
queries_button.pack(pady=5)

# Create a button to open the insert data screen
insert_button = ttk.Button(root, text="Insert Data", command=open_insert_screen)
insert_button.pack(pady=5)

# Run the Tkinter event loop
root.mainloop()

# Close the database connection
connection.close()
